# 任务三 会员用户画像

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [10]:
data = pd.read_csv('mb_sell.csv')

## 构建会员用户基本特征标签

In [88]:
data['csrq'] = pd.to_datetime(data['csrq'])
data['dtime'] = pd.to_datetime(data['dtime'])
data['djsj'] = pd.to_datetime(data['djsj'])

In [13]:
data['age'] = data['csrq'].apply(lambda x: 2020 - x.year)
data['mb_time'] = data['djsj'].apply(lambda x: 2020 - x.year)

In [104]:
data['month'] = data['dtime'].dt.month.replace(12,0)
data['quarter'] = pd.cut(data['dtime'].dt.month,bins=[0,2,5,8,11],labels = ['冬季','春季','夏季','秋季'])
data['time_lab'] = pd.cut(data['dtime'].dt.hour,
                          bins=[0,5,11,13,17,24],labels = ['凌晨','早上','中午','下午','晚上'])

In [134]:
data['max_time'] = data[data['dtime']].max()

In [135]:
data.head()

,kh,csrq,xb,djsj,dtime,spbm,sl,sj,je,spmc,...,syjh,djh,gzbm,gzmc,age,mb_time,month,quarter,time_lab,max_time
0,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,297.07,499.0,297.07,WMF D无,...,101.0,7cd8,7296.0,WMF 柜,53,16,9,秋季,早上,2018-01-03 21:27:28.686
1,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,11089.93,12288.0,11089.93,WMF D无,...,101.0,7cd8,7296.0,WMF 柜,53,16,9,秋季,早上,2018-01-03 21:27:28.686
2,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2017-02-28 17:17:35.533,252403ef,598.00,598.0,598.00,双立人商品 F无,...,102.0,ed0a,7242.0,双立人柜,53,16,2,冬季,下午,2018-01-03 21:27:28.686
3,34c2dc93,1986-07-18,0.0,2007-04-19 00:00:00.000,2017-11-13 13:36:45.656,4708e0bb,260.00,260.0,260.00,植村秀三色眼影/眼线液/眉笔.,...,97.0,c3f4,8156.0,Shu Uemura,34,13,11,秋季,中午,2018-01-03 21:27:28.686
4,828aa9b4,1951-12-29,0.0,2004-09-24 16:39:08.716,2017-05-17 11:30:39.080,a4809307,1288.00,1288.0,1288.00,酷彩F件,...,102.0,95b0,7300.0,Le Creuset(酷彩),69,16,5,春季,早上,2018-01-03 21:27:28.686


In [146]:
kh_num = list(set(data['kh']))
#kh_test = kh_num[1:50]
kh_test = kh_num

In [147]:
def get_labs(kh):
    order = data[data['kh'] == kh]
    # 基本特征标签
    xb = order.iloc[0]['xb'] # 性别
    if xb == 0.0:
        xb = '女'
    elif xb == 1.0:
        xb = '男'
    mb_time = order.iloc[0]['mb_time'] #入会时长（天）
    age = order.iloc[0]['age'] # 年龄
    
    # 业务特征标签
    integral = np.sum(order['jf']) # 积分
    cost_lev = np.sum(order['je']) # 消费水平
    cost_freq = len(set(order['dtime'].apply(lambda x:(x.year,x.month,x.day,x.hour)))) # 消费频次
    max_time = np.max(order['dtime'])
    
    # 兴趣特征标签
    gzmc = order['gzmc']
    gzmc_like = gzmc.value_counts()
    if len(gzmc_like) > 5:
        gzmc_labs = list(gzmc_like.index[:5])
    else:
        gzmc_labs = list(gzmc_like.index)  
        
    # 季节偏好,选取最高频的购物季节
    jj = list(set(order['quarter'].values)) 
    gwjj_list = list(order['quarter'].values)
    jjph = jj[0]
    jjcs = gwjj_list.count(jjph)
    for i in jj:
        if gwjj_list.count(i) > jjcs:
            jjph = i
            jjcs = gwjj_list.count(i)
            
    time = list(set(order['time_lab'].values)) 
    gwtime_list = list(order['time_lab'].values)
    timeph = time[0]
    timecs = gwtime_list.count(timeph)
    for i in jj:
        if gwtime_list.count(i) > timecs:
            jjph = i
            timecs = gwtime_list.count(i)
    return xb,age,mb_time,integral,cost_lev,cost_freq,max_time,gzmc_labs,jjph,timeph

## 建立用户画像

In [148]:
性别 = []
年龄 = []
入会时长 = []
积分 = []
消费水平 = []
消费频率 = []
最后一次消费时间 = []
消费偏好 = []
季节消费偏好 = []
时间段消费偏好 = []
i = 0
for kh in kh_test:
    xb,age,mb_time,integral,cost_lev,cost_freq,max_time,gzmc_labs,jjph,timeph = get_labs(kh)
    性别.append(xb)
    年龄.append(age)
    入会时长.append(mb_time)
    积分.append(integral)
    消费水平.append(cost_lev)
    消费频率.append(cost_freq)
    最后一次消费时间.append(max_time)
    消费偏好.append(gzmc_labs)
    季节消费偏好.append(jjph)
    时间段消费偏好.append(timeph)

In [149]:
labels = pd.DataFrame()
labels['卡号'] = kh_test
labels['性别'] = 性别
labels['年龄'] = 年龄
labels['入会时长(年)'] = 入会时长
labels['积分'] = 积分
labels['消费金额'] = 消费水平
labels['消费频率'] = 消费频率
labels['最后一次消费时间'] = 最后一次消费时间
labels['购物类型偏好'] = 消费偏好
labels['购物季节偏好'] = 季节消费偏好
labels['购物时间偏好'] = 时间段消费偏好

In [150]:
labels.head()

,卡号,性别,年龄,入会时长(年),积分,消费金额,消费频率,最后一次消费时间,购物类型偏好,购物季节偏好,购物时间偏好
0,c5935c98,女,50,4,2659.0,2659.0,2,2015-07-08 20:22:16.640,"[LUNA柜, ARMANI COLLEZIONI]",夏季,晚上
1,9ae09d4d,男,46,4,310.0,310.0,1,2017-07-20 20:18:30.046,[阿玛尼GIORGIO ARMANI],夏季,晚上
2,e2bd28ff,女,39,6,4375.0,4375.0,1,2015-02-02 16:26:30.953,"[欧舒丹柜, 兰芝柜]",冬季,下午
3,7271dd79,女,54,6,1078.0,1078.0,2,2017-11-19 15:48:59.533,"[Rime, MARJA KURKI]",秋季,下午
4,76fd7194,女,35,3,1775.0,1775.0,2,2017-02-08 11:01:39.906,"[Shu Uemura, LANCOME]",冬季,下午


In [128]:
labels['消费金额'].max()

3281897.8500000006

In [151]:
labels['消费水平'] = pd.cut(labels['消费金额'],bins=[0,10000,99999,3300000],labels = ['低消费','中等消费','高消费'])

In [152]:
labels.head()

,卡号,性别,年龄,入会时长(年),积分,消费金额,消费频率,最后一次消费时间,购物类型偏好,购物季节偏好,购物时间偏好,消费水平
0,c5935c98,女,50,4,2659.0,2659.0,2,2015-07-08 20:22:16.640,"[LUNA柜, ARMANI COLLEZIONI]",夏季,晚上,低消费
1,9ae09d4d,男,46,4,310.0,310.0,1,2017-07-20 20:18:30.046,[阿玛尼GIORGIO ARMANI],夏季,晚上,低消费
2,e2bd28ff,女,39,6,4375.0,4375.0,1,2015-02-02 16:26:30.953,"[欧舒丹柜, 兰芝柜]",冬季,下午,低消费
3,7271dd79,女,54,6,1078.0,1078.0,2,2017-11-19 15:48:59.533,"[Rime, MARJA KURKI]",秋季,下午,低消费
4,76fd7194,女,35,3,1775.0,1775.0,2,2017-02-08 11:01:39.906,"[Shu Uemura, LANCOME]",冬季,下午,低消费


In [154]:
labels.to_csv('mb_fx.csv',index = False)